# Profiling: Finding bottlenecks in Python programs
**Contributors**: Simon Funke

*"First make it work. Then make it right. Then make it fast."* 
  
  Kent Beck

## Profiling allows us to measure resources used by sections of the program. 

Typical resources of interest are
* CPU
* Memory
* Disk I/O
* Network I/O
* ...

We focus on **CPU** profiling only.

## CPU: start simple, switch to more complex techniques if needed!

1. Manual timing
2. `timeit` module
3. `cprofile` module

## Case study: filling a grid with point values

* Consider a rectangular 2D grid
<center>![lattice](pdf/grid_lattice.svg "Python")Grid lattice</center>
* A NumPy array `a[i,j]` holds values at the grid points

# An implementation

In [1]:
from numpy import *

class Grid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        lx = size(self.xcoor)
        ly = size(self.ycoor)
        a = zeros((lx,ly))

        for i in range(lx):
            x = self.xcoor[i]
            for j in range(ly):
                y = self.ycoor[j]
                a[i,j] = f(x, y)
        return a

# Usage

Create a new grid:

In [2]:
g = Grid2D(dx=0.001, dy=0.001)

Computing grid values:

In [3]:
def myfunc(x, y):
    return sin(x*y) + y

print("Computing values...")
a = g.gridloop(myfunc)
print("done")

Computing values...
done


# Timing

Use `time.time()` to measure the time spend in a code section.
```python
import time
t0 = time.time()
# execute code here
t1 = time.time()
print("Runtime: {}".format(t1-t0))
```
  

**Tips**:
* Simple statements should be placed in a loop.
* Make sure to have no other expensive programs running.
* Run the tests several times, choose the fastest. **Why?**

## Python challenge

1. Download [grid.py](https://raw.githubusercontent.com/UiO-INF3331/UiO-INF3331.github.io/crash-course/lectures/profiling/code/grid.py) and run it. Which parts of the program might be slow (consider the main functions first)?
2. Time the suspected slow code lines and note down how long it takes.

## Timing of `grid.py`

Two parts that could potentially be slow: 
1. The initialisation `Grid2D(dx=0.001, dy=0.001)`
2. Calling the `g.gridloop(myfunc)` function.

We time these two parts separately to figure out how much time is spend in each.

### Timing the Grid2D initialisation

In [4]:
import time

for i in range(1, 4):
    t0 = time.time()
    g = Grid2D(dx=0.001, dy=0.001)
    t1 = time.time()
    print("Experiment {}, CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1, CPU time: 0.0003524 s
Experiment 2, CPU time: 0.000164 s
Experiment 3, CPU time: 3.886e-05 s
Done


### Timing the `gridloop` function

In [5]:
import time

for i in range(1, 4):
    t0 = time.time()
    g.gridloop(myfunc)
    t1 = time.time()
    print("Experiment {}. CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1. CPU time: 2.236 s
Experiment 2. CPU time: 2.68 s
Experiment 3. CPU time: 2.73 s
Done


$=>$ The gridloop function is the cause of the slow execution!

# The *timeit* module

## The *timeit* module (1)

The `timeit` module provides an convenient way for measuring the CPU time of small code snippets.

In [6]:
import timeit
timeit.timeit(stmt="a+=1", setup="a=0")  

0.1685657119996904

By default the **accumulated time** for 1,000,000 statements is returned. This number can be changed with the `number` keyword:

In [7]:
timeit.timeit(stmt="a+=1",setup="a=0", number=1)

2.2050007828511298e-06

Use `timeit.repeat` to repeat the experiment multiple times:

In [8]:
timeit.repeat(stmt="a+=1",setup="a=0", number=10000, repeat=3)

[0.0007999480003491044, 0.0007958939986565383, 0.0011175279996678]

## Wall-clock time vs CPU time


In [9]:
timeit.timeit(stmt="sleep(1)", setup="from time import sleep", number=1, 
              timer=time.time)  # Wall-clock time

1.0017197132110596

In [10]:
timeit.timeit(stmt="sleep(1)", setup="from time import sleep", number=1, 
              timer=time.clock)  # CPU time

0.0015919999999987056

# Timing user-defined functions requires a trick

`timeit` isolates the global namespace, so we need to pass the functions through the setup routine:

In [20]:
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", number=1)

[1.973618121999607, 1.9566386259994033, 1.9746652839985472]

## IPython's `%timeit` magic

IPython provides the `%timeit` magic to simplify timing code in the IPython shell

In [23]:
%timeit g.gridloop(myfunc) 

1 loop, best of 3: 1.95 s per loop


The `timeit` magic determins automatically how often to repeat the experiment.

In [24]:
%timeit?

## Python challenges

1. Repeat the timings for the `grid.py` program with the `timings` module. Time both the wall-clock time and the CPU time. How much do they differ? 
2. Repeat the timing experiment 100 times and create a histogram of the results (use the `matplotlib.pyplot.hist` function of the result).

## Profile

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

There are two main (deterministic) profiling modules in Python

* `profile`, written purely in Python
* `cProfile` is a compatible C implementation with smaller performance impact than `profile`. In particular overhead calibaration not required. 

We will be using `cProfile`.

The statistics from both can be formatted into reports via the `pstats` module.    

## Using the `cProfile` module

* As a script: `python -m cProfile script.py
`

* As a module:

In [12]:
import cProfile
pr = cProfile.Profile()
res = pr.run("g.gridloop(myfunc)")  # res contains the statistics
pr.dump_stats("gridloop.prof")  # Dump statistics to file for use with pstats

## Profile a function call while keeping its return value

`pr.run("foo()")` returns an object with the profile statistics. If one needs the return value of `foo()`, use `pr.runcall` instead:

In [13]:
 pr.runcall(g.gridloop, myfunc)

array([[  0.00000000e+00,   1.00000000e-03,   2.00000000e-03, ...,
          9.98000000e-01,   9.99000000e-01,   1.00000000e+00],
       [  0.00000000e+00,   1.00100000e-03,   2.00200000e-03, ...,
          9.98998000e-01,   9.99999000e-01,   1.00100000e+00],
       [  0.00000000e+00,   1.00200000e-03,   2.00400000e-03, ...,
          9.99995999e-01,   1.00099800e+00,   1.00200000e+00],
       ..., 
       [  0.00000000e+00,   1.99799983e-03,   3.99599867e-03, ...,
          1.83730522e+00,   1.83884738e+00,   1.84038870e+00],
       [  0.00000000e+00,   1.99899983e-03,   3.99799867e-03, ...,
          1.83784738e+00,   1.83938924e+00,   1.84093026e+00],
       [  0.00000000e+00,   1.99999983e-03,   3.99999867e-03, ...,
          1.83838870e+00,   1.83993026e+00,   1.84147098e+00]])

## Getting runtime statistics

* We can view some runtime statistics with

In [14]:
res.print_stats()

         2004014 function calls in 4.732 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    1.546    0.773    4.732    2.366 <ipython-input-1-af8978036994>:11(gridloop)
  2004002    3.183    0.000    3.183    0.000 <ipython-input-3-a9b58edea67e>:1(myfunc)
        1    0.000    0.000    2.424    2.424 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 fromnumeric.py:2659(size)
        1    0.000    0.000    2.424    2.424 {built-in method builtins.exec}
        2    0.002    0.001    0.002    0.001 {built-in method numpy.core.multiarray.zeros}
        2    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Statistics header definitions
* **ncalls**: number of calls
* **tottime**: total time spent in the given function excluding time made in calls to sub-functions
* **percall**: tottime divided by ncalls
* **cumtime**: cumulative time spent in this and all subfunctions
* **percall**: cumtime divided by ncalls
* **filename:lineno(function)**: information about the function

## The `pstats` module

There are many ways to view profiling data.

The module `pstats` provides the class `Stats` for creating profiling reports:

In [15]:
import pstats
stats = pstats.Stats("gridloop.prof")
stats.sort_stats("time")
stats.print_stats(3)

Sun Jan 29 16:14:50 2017    gridloop.prof

         1002008 function calls in 2.424 seconds

   Ordered by: internal time
   List reduced from 7 to 3 due to restriction <3>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1002001    1.633    0.000    1.633    0.000 <ipython-input-3-a9b58edea67e>:1(myfunc)
        1    0.790    0.790    2.424    2.424 <ipython-input-1-af8978036994>:11(gridloop)
        1    0.001    0.001    0.001    0.001 {built-in method numpy.core.multiarray.zeros}




* The method `sort_stats(key, *keys)` is used to sort future output.
* Common used keys: `'calls', 'cumulative', 'time'`.
* The argument to `print_stats` limits the view to the N most significant results.

## Back to our case-study: What have we learned so far?

The total runtime (with `profile`) is 8.1s. Biggest contributors:
   1. `gridloop` contributes ca 41% of the total runtime.
   2. `myfunc` calls contributes ca 55% of the total runtime.

* `myfunc` is fairly straight-forward
  ```python
  def myfunc(x, y):
    return sin(x*y) + y
  ```
  Might be difficult to improve.
* What about `gridloop`?

## Recall that, `gridloop` was a function of the form

```python
def gridloop(self, f):
    lx = size(self.xcoor)
    ly = size(self.ycoor)
    a = zeros((lx,ly))

    for i in range(lx):
        x = self.xcoor[i]
        for j in range(ly):
            y = self.ycoor[j]
             a[i,j] = f(x, y)
    return a
```

It would be useful to see how much time is spend in each line!

## Line by line profiling

The `line_profiler` inspects the time spend in each line of a Python function. 

## Usage

1. Install with `pip install line_profiler`
2. "Decorate" the function of interest with `@profile`:
    ```python
    @profile
    def gridloop(func):
        # ...
    ```
3. Run line profiler with:
    ```bash
    kernprof -l -v grid2d_lineprofile.py
    ```

## Demo

In [16]:
!kernprof -l -v grid2d_lineprofile.py

Could not find script grid2d_lineprofile.py


**Conclusion:** Most of the time is spend in loops and indexing.

## A vectorised Grid2D implementation

In [17]:
class VectorisedGrid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        return f(self.xcoor[:,None], self.ycoor[None,:])  # Vectorized grid evaluation 

## Timing the vectorised Grid2D

In [18]:
vg = VectorisedGrid2D(dx=0.001, dy=0.001)
timeit.repeat(stmt="vg.gridloop(myfunc)", setup="from __main__  import vg, myfunc", repeat=5, number=1)

[0.03718072599986044,
 0.03210592799950973,
 0.03234826200059615,
 0.037420828000904294,
 0.03362453099907725]

In [19]:
g = Grid2D(dx=0.001, dy=0.001)
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

[2.0139896309992764,
 1.9716419559990754,
 2.0408604399999604,
 2.0111127870004566,
 2.0976008960005856]

**Vectorization yields a 70 times speed improvement!**

### Python challenge

